目的：
1. 绘制 IMU 数据，并左右展示数据
步骤：
1. 导入包
2. 定义路径
3. 编写函数，批量运行

In [14]:
#导入包
import os
import re
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
#确定路径，并切换
path = input("请输入数据的路径")
os.chdir(path)
print("当前路径为：{}".format(os.getcwd()))

当前路径为：/Users/wangshuaibo/Documents/北京悦动/脑卒中项目/阿帅笔记/测试数据及分析结果/20230418鞋垫预实验/20230419/InShoe


In [16]:
#新建 IMU_plot  文件夹
dir_plot_name = "IMU_plot"

if not os.path.exists(dir_plot_name):
    os.mkdir(dir_plot_name)

if os.path.exists(dir_plot_name):
    print("已经成功新建 IMU_plot")

已经成功新建 IMU_plot


In [18]:
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation

def transform_euler_angles(quaternion):
    """
    使用四元数转换欧拉角，并将其转换为特定形式的欧拉角。
    使用阿鲁比小蓝块 IMU 数据进行验证，没有进行鞋垫姿态传感器的验证
    
    Args:
        quaternion (list): 长度为4的Series，表示四元数。
    
    Returns:
        pd.Series: 长度为3的Series，表示经过转换的欧拉角。
    """
    # 使用四元数转换欧拉角
    rotation = Rotation.from_quat(quaternion)
    euler_angles = rotation.as_euler("zyx", degrees=True)
    
    # 将欧拉角转换为特定形式
    # 仅使用 transform 转化，得到的数据不正确，需要进一步转化
    # 第一个角度不需要变化，第二个角度前面加负数，第三个是和180互补加负数
    # 欧拉角的角度范围应该是 -180～180，但是现在数据范围大于 180
    transformed_angles = np.round([euler_angles[0], -euler_angles[1], -(180 - euler_angles[2])], 6)
    # transformed_angles = np.round([euler_angles[0], euler_angles[1], euler_angles[2]], 6)
    
    return pd.Series(transformed_angles)

In [19]:
#确定图片储存的路径以及名称
def plot_IMU_data(path,dir_plot_name,file,start_index,end_index):
    regex = r"(.*)_(.*)_(.*)"
    mysearch = re.search(regex,file)
    if mysearch:
        plot_name = mysearch.group(2) + "-" + dir_plot_name
    path_plot = os.path.join(path,dir_plot_name,plot_name)

    df = pd.read_csv(file)
    euler_cols = ["euler_x","euler_y","euler_z"]
    g_cols = ["gx","gy","gz"]
    a_cols = ["ax","ay","az"]
    q_cols = ["qx","qy","qz","qw"]
    df[euler_cols] = df[q_cols].apply(transform_euler_angles,axis=1)

    df_sel = df.loc[start_index:end_index-1]
    L_data = df_sel[df_sel["part" ]== "L"]
    R_data = df_sel[df_sel["part" ]== "R"]
    euler_cols = ["euler_x","euler_y","euler_z"]
    g_cols = ["gx","gy","gz"]
    a_cols = ["ax","ay","az"]
    q_cols = ["qx","qy","qz","qw"]


    #创建一个包含多个子图的 Figure 对象和 AxesSubplot
    fig, axs = plt.subplots(nrows=3, ncols=2,figsize=(12,6))

    #调整子图的位置和大小
    plt.subplots_adjust(wspace=0.2, hspace=0.4)
    fig.set_size_inches(10,8)

    plt.rcParams['font.sans-serif'] = ['Heiti TC', 'SimHei', 'FangSong']  # 汉字字体,优先使用楷体，如果找不到楷体，则使用黑体
    plt.rcParams['font.size'] = 12  # 字体大小
    plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号

    #绘制左右脚数据
    L_data.loc[:,euler_cols].plot(ax=axs[0,0],title="左脚欧拉角数据（°）")
    R_data.loc[:,euler_cols].plot(ax=axs[0,1],title="右脚欧拉角数据（°）")

    L_data.loc[:,a_cols].plot(ax=axs[1,0],title="左脚加速度数据")
    R_data.loc[:,a_cols].plot(ax=axs[1,1],title="右脚加速度数据")

    L_data.loc[:,g_cols].plot(ax=axs[2,0],title="左脚角速度数据")
    R_data.loc[:,g_cols].plot(ax=axs[2,1],title="右脚角速度数据")

    # 添加注释，解释代码的目的和功能
    plt.suptitle(plot_name, fontsize=16)
    fig.text(0.5, 0.02, "时间", ha="center")
    fig.text(0.08, 0.5, "数据值", va="center", rotation="vertical")

    #保存图片
    plt.savefig(path_plot+".png")
    plt.close()

In [20]:

for file in os.listdir(path):
    if ".csv" in file:
        # 默认索引全部数据
        df = pd.read_csv(file)
        end_index = df.shape[0]
        start_index = 0

        #绘图
        plot_IMU_data(path,dir_plot_name,file,start_index,end_index)

In [15]:
import numpy as np
from scipy.spatial.transform import Rotation

df[["Eulerx","Eulery","Eulery"]] = df[["qw","qx","qy","qz"]].apply(transform_euler_angles,axis=1)

In [16]:
df[["Eulerx","Eulery","Eulery"]]

,Eulerx,Eulery,Eulery
0,1.757530,-197.542088,-197.542088
1,4.727731,-212.603832,-212.603832
2,0.407102,-315.756699,-315.756699
3,5.394189,-346.083451,-346.083451
4,0.361573,-315.773540,-315.773540
...,...,...,...
531,5.498705,-340.382987,-340.382987
532,-0.139663,-320.898346,-320.898346
533,5.494758,-340.384215,-340.384215
534,-0.209574,-320.877157,-320.877157


------
hwshuai_走6步路-鞋垫2-1-模块41AB-V4.1-02_202304171229507081.csv
/Users/wangshuaibo/Documents/北京悦动/脑卒中项目/阿帅笔记/测试数据及分析结果/20230417医院预实验前准备测试/IMU_plot
------
------
hwshuai_跺脚走6步路-鞋垫2-1-模块41AB-V4.1-01_202304171229507102.csv
------
.DS_Store
------
hwshuai_走6步路-鞋垫2-1-模块41AB-V4.1-03_202304171229507060的副本.csv
------
hwshuai_跺脚走6步路-鞋垫2-1-模块41AB-V4.1-04_202304171502508920.csv
------
hwshuai_走6步路-鞋垫2-1-模块41AB-V4.1-03_202304171229507060.csv


In [73]:
for dirpath, dirnames, filenames in os.walk(path):
    #print(dirpath)
    #print("-----")
    #print(dirnames)
    print(f"dirpath:\n{dirpath}\n")
    print(f"dirnames:\n{dirpath}\n")
    print(f"dirnames:\n{filenames}")
    print("-----\n\n")

dirpath:
/Users/wangshuaibo/Documents/北京悦动/脑卒中项目/阿帅笔记/测试数据及分析结果/20230417医院预实验前准备测试

dirnames:
/Users/wangshuaibo/Documents/北京悦动/脑卒中项目/阿帅笔记/测试数据及分析结果/20230417医院预实验前准备测试

dirnames:
['hwshuai_走6步路-鞋垫2-1-模块41AB-V4.1-02_202304171229507081.csv', 'hwshuai_跺脚走6步路-鞋垫2-1-模块41AB-V4.1-01_202304171229507102.csv', '.DS_Store', 'hwshuai_走6步路-鞋垫2-1-模块41AB-V4.1-03_202304171229507060的副本.csv', 'hwshuai_跺脚走6步路-鞋垫2-1-模块41AB-V4.1-04_202304171502508920.csv', 'hwshuai_走6步路-鞋垫2-1-模块41AB-V4.1-03_202304171229507060.csv']
-----


dirpath:
/Users/wangshuaibo/Documents/北京悦动/脑卒中项目/阿帅笔记/测试数据及分析结果/20230417医院预实验前准备测试/IMU_plot

dirnames:
/Users/wangshuaibo/Documents/北京悦动/脑卒中项目/阿帅笔记/测试数据及分析结果/20230417医院预实验前准备测试/IMU_plot

dirnames:
['跺脚走6步路-鞋垫2-1-模块41AB-V4.1-01-IMU_plot.png', '走6步路-鞋垫2-1-模块41AB-V4.1-02-IMU_plot.png', '走6步路-鞋垫2-1-模块41AB-V4.1-03-IMU_plot.png', '跺脚走6步路-鞋垫2-1-模块41AB-V4.1-04-IMU_plot.png']
-----




In [50]:
filenames

['跺脚走6步路-鞋垫2-1-模块41AB-V4.1-01-IMU_plot.png',
 '走6步路-鞋垫2-1-模块41AB-V4.1-02-IMU_plot.png',
 '走6步路-鞋垫2-1-模块41AB-V4.1-03-IMU_plot.png',
 '跺脚走6步路-鞋垫2-1-模块41AB-V4.1-04-IMU_plot.png']